In [4]:
!pip install pyspark==3.0.1 py4j==0.10.9 
# from pyspark.sql import SparkSession
# import numpy as np
from pyspark.sql import SparkSession
import numpy as np

In [6]:
#create instance of spark class 
spark=SparkSession.builder.appName('Questions').getOrCreate()

Exception: Java gateway process exited before sending its port number

In [ ]:
spark

In [ ]:
emp = spark.read.csv('employee.csv',header='true', 
                      inferSchema='true')
emp.show(5)

In [ ]:
dept = spark.read.csv('department.csv',header='true', 
                      inferSchema='true')
dept.show(5)

In [ ]:
location = spark.read.csv('location.csv',header='true', 
                      inferSchema='true')
location.show(5)

In [ ]:
countries = spark.read.csv('country.csv',header='true', 
                      inferSchema='true')
countries.show(5)

**1. Get list of employess along with their department details**

In [ ]:
emp_dept =  emp.join(dept,emp["department_id"] ==  dept["department_id"],"left")
emp_dept.show(5)

**2. Analysis Employee Department**

In [ ]:
emp_dept.groupBy('department_name').count().show()

In [ ]:
department_count = emp_dept.select('department_name').toPandas()
department_count.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plot_categorical('department_name',department_count)

**Conclusion**

* Majority of employess(around 17.5%) work in Shipping Department
* Administration, Human Resources and Public Relations are Departments having least number of Employees working
* Finance, Sales and Purchasing departments have equal percentage of employees employeed 

**3. Get Address of Administration and Marketing Department**

In [ ]:
department_locations = dept.join(location,dept["location_id"] ==  location["location_id"],"left")

In [ ]:
countries_department_locations = department_locations.join(countries,department_locations["country_id"] ==  countries["country_id"],"left")
countries_department_locations.show(1)

**4. Get Employees who work in USA as Administration department**

In [ ]:
employees_department_locations = emp.join(countries_department_locations,emp["department_id"] ==  countries_department_locations["department_id"],"left")
employees_department_locations.show(1)

**5. How many employees work in Canada ?**

In [ ]:
employees_department_locations.collect()

In [ ]:
employees_department_locations.groupBy('country_name').count().show()


**Only two people works in canada**

**6. Get Complete Address of Employees**

In [ ]:
employees_department_locations.printSchema()

In [ ]:
from pyspark.sql.functions import concat , col , lit
employees_department_locations.withColumn("Address",concat(col("street_address"), lit(","), col("city"), lit(","), col("postal_code"), lit(","), col("state_province") , lit(","), col("country_name"))).select('Address').show()

**7. Convert Employee's First name and last name to lower case letters and concatinate them**

In [ ]:
def convertCase(string):
    return string.lower()

In [ ]:
from pyspark.sql.functions import udf , StringType
convertUDF = udf(lambda z: convertCase(z),StringType())

In [ ]:
emp.select(convertUDF(col("first_name")).alias("first_name"), convertUDF(col("last_name")).alias("last_name") ).withColumn("Name",concat(col("first_name"), lit(" "), col("last_name"))).select('Name').show()

**8. Get Hightest paid jobs**

In [ ]:
from pyspark.sql.functions import max
employees_department_locations.select(max('salary')).collect()[0][0]

In [ ]:
data = employees_department_locations.select('*')

In [ ]:
from pyspark.sql import SQLContext
data.registerTempTable("data")
sql_context = SQLContext(spark)
sql_context.sql(f"SELECT department_name from data WHERE salary == {employees_department_locations.select(max('salary')).collect()[0][0]}").collect()[0][0]

Hightest paid job is Executive